In [1]:
import pandas as pd
import magic 
import re
from datetime import datetime


In [2]:
magic.from_file('results.csv')


'ASCII text, with CRLF line terminators'

In [3]:
df = pd.read_csv('results.csv',encoding= 'ISO-8859-1')
df.iloc[23763:23766] #Intervalo para análise do problema encontrado no PBI


,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
23763,23768,988,815,10,11.0,8,7.0,7,7,6.0,55,+1:32.062,5746124.0,52.0,11.0,01:42.7,194.708,1
23764,23769,988,839,10,31.0,9,8.0,8,8,4.0,55,+1:38.911,5752973.0,47.0,10.0,01:42.6,01:42.6,1
23765,23770,988,4,1,14.0,11,9.0,9,9,2.0,54,NaN,NaN,26.0,4.0,01:43.4,193.41,11


In [4]:
#Função para transformar tempo (str) em tempo em segundos (float)
def strtime_to_sec(tempo_str):
    #print(f"Tempo em {type(tempo_str)}: {tempo_str}")

    tempo =  datetime.strptime(tempo_str,'%M:%S.%f')
    #print(f"Tempo em {type(tempo)}: {tempo}")

    tempo_int = int(tempo.strftime("%M%S%f"))
    #print(f"Tempo em {type(tempo2)}: {tempo2}")

    minutos = (tempo_int//100_000_000)*60
    segundos = tempo_int%100_000_000//1_000_000
    milisegundos = tempo_int%1_000_000/1_000_000

    tempo_total = minutos+segundos+milisegundos
    return tempo_total


In [5]:
#Função para corrigir a velocidade
def cor_vel(indice):
    tempo1 = strtime_to_sec(df['fastestLapTime'].iloc[indice])

    tempo2 = strtime_to_sec(df['fastestLapTime'].iloc[indice-1])

    vel2 = float(df['fastestLapSpeed'].iloc[indice-1])

    vel1 = round((tempo1 * vel2)/tempo2,3)

    df.loc[indice,'fastestLapSpeed'] = vel1

In [6]:
v_correta = 0
v_corrigida = 0
indices_corrigidos = []
for i in df.itertuples():
    try:
        float(i.fastestLapSpeed)
        v_correta = v_correta + 1
    except:
        cor_vel(i.Index)
        indices_corrigidos.append(i.Index)
        v_corrigida = v_corrigida + 1

print(f"velocidades corretas: {v_correta}")
print(f"velocidades corrigidas: {v_corrigida}")
print(f"Indices corrigidos: {indices_corrigidos}")

df.iloc[23763:23766] #Conferindo intervalo do problema encontrado no PBI

velocidades corretas: 23776
velocidades corrigidas: 1
Indices corrigidos: [23764]


,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
23763,23768,988,815,10,11.0,8,7.0,7,7,6.0,55,+1:32.062,5746124.0,52.0,11.0,01:42.7,194.708,1
23764,23769,988,839,10,31.0,9,8.0,8,8,4.0,55,+1:38.911,5752973.0,47.0,10.0,01:42.6,194.518,1
23765,23770,988,4,1,14.0,11,9.0,9,9,2.0,54,NaN,NaN,26.0,4.0,01:43.4,193.41,11


In [7]:
df.to_csv('results_v1.csv',encoding= 'ISO-8859-1',index=False)